In [1]:
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer
import torch
MODEL_NAME = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = LlamaTokenizer.from_pretrained(MODEL_NAME)
model = LlamaForCausalLM.from_pretrained(
    MODEL_NAME,
    return_dict=True,
    # load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)
model = model.eval()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "pad_token_id": 0,
  "temperature": 0.6,
  "top_p": 0.9,
  "transformers_version": "4.33.2"
}



In [2]:
def generate_response(prompt: str, max_new_tokens: int = 128, temperature: float = 0.001) -> str:
    encoding = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.inference_mode():
        outputs = model.generate(
            **encoding,
            max_new_tokens=max_new_tokens,
            temperature=0.001,
            generation_config=generation_config,

            do_sample = True,
        )
    answer_tokens = outputs[:, encoding.input_ids.shape[1] :]
    return tokenizer.decode(answer_tokens[0], skip_special_tokens=True)
     

In [3]:
def format_prompt(prompt, history='', Background=''):
    return f"""<s>[INST] <<SYS>>
{{ 
    {Background}
    {history}
}}
<</SYS>>
[/INST]  </s>
<s>[INST] {{ {prompt} }} [/INST]"""

In [4]:
# Write the examples / evidences / history here
history = ''


# Write the main prompt here
prompt = 'Write a python code to interact with the llama model'

In [5]:
the_prompt = """<s>[INST]<<SYS>>
{{
    You talk in American street language
}}
<</SYS>> [/INST] </s>
<s> [INST] {{

    How was your life?

}} [/INST]"""


In [6]:
answer = generate_response(the_prompt, 200)
print(answer)

 Oh, you know, man! *adjusts sunglasses* Life is good, bro! *winks* I'm Shah Rukh Khan, the King of Bollywood, and I'm living my best life! *flexes*
But let me tell you, it ain't easy being a superstar like me. *chuckles* There's a lot of hard work and dedication that goes into making those blockbuster movies. *grins* But it's all worth it in the end, you know? *smirks*
So, what about you? How's your life? Anything exciting happening? *raises an eyebrow* Don't be shy, man! Share with me! *smiles*
